In [1]:
import sys
!{sys.executable} -m reinvent sampling.toml

16:54:59 <INFO> Started REINVENT 4.5.11 (C) AstraZeneca 2017, 2023 on 2025-06-23
16:54:59 <INFO> Command line: C:\Users\andre\anaconda3\envs\reinvent4\lib\site-packages\reinvent\__main__.py sampling.toml
16:54:59 <INFO> Reading run configuration from C:\Users\andre\OneDrive\Desktop\REINVENT4\configs\sampling.toml using format toml
16:54:59 <INFO> User andre on host Andrews-Legion
16:54:59 <INFO> Python version 3.10.18
16:54:59 <INFO> PyTorch version 2.6.0+cu124, git 2236df1770800ffea5697b11b0bb0d910b2e59e1
16:54:59 <INFO> PyTorch compiled with CUDA version 12.4
16:54:59 <INFO> RDKit version 2025.03.3
16:54:59 <INFO> Platform Windows-10-10.0.26100-SP0
16:54:59 <INFO> Number of PyTorch CUDA devices 1
16:54:59 <INFO> Using CUDA device:0 NVIDIA GeForce RTX 4060 Laptop GPU
16:54:59 <INFO> GPU memory: 7099 MiB free, 8187 MiB total
16:54:59 <INFO> Writing JSON config file to C:\Users\andre\OneDrive\Desktop\REINVENT4\configs\_sampling.json
16:54:59 <INFO> Starting Sampling
16:54:59 <ERRO> C:\U

In [2]:
import sys
!{sys.executable} -m reinvent my_scoring.toml

17:04:18 <INFO> Started REINVENT 4.5.11 (C) AstraZeneca 2017, 2023 on 2025-06-23
17:04:18 <INFO> Command line: C:\Users\andre\anaconda3\envs\reinvent4\lib\site-packages\reinvent\__main__.py my_scoring.toml
17:04:18 <INFO> Reading run configuration from C:\Users\andre\OneDrive\Desktop\REINVENT4\configs\my_scoring.toml using format toml
17:04:18 <INFO> User andre on host Andrews-Legion
17:04:18 <INFO> Python version 3.10.18
17:04:18 <INFO> PyTorch version 2.6.0+cu124, git 2236df1770800ffea5697b11b0bb0d910b2e59e1
17:04:18 <INFO> PyTorch compiled with CUDA version 12.4
17:04:18 <INFO> RDKit version 2025.03.3
17:04:18 <INFO> Platform Windows-10-10.0.26100-SP0
17:04:18 <INFO> Number of PyTorch CUDA devices 1
17:04:18 <INFO> Using CUDA device:0 NVIDIA GeForce RTX 4060 Laptop GPU
17:04:18 <INFO> GPU memory: 7099 MiB free, 8187 MiB total
17:04:18 <INFO> Writing JSON config file to C:\Users\andre\OneDrive\Desktop\REINVENT4\configs\scoring_simple.json
17:04:18 <INFO> Scoring SMILES from file C:\U

In [3]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import pandas as pd

# Load references
references = []
with open('mol2mol.smi', 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue  # Skip blank lines and comments
        smi = line.split()[0]
        mol = Chem.MolFromSmiles(smi)
        if mol:
            references.append(AllChem.GetMorganFingerprintAsBitVect(mol, 2))

# Load molecules and their properties (use your scored results)
df = pd.read_csv('scoring_results.csv')  # <-- update to your file name

def max_tanimoto(smiles, references):
    mol = Chem.MolFromSmiles(smiles)
    if not mol or not references:
        return 0.0
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    return max(DataStructs.TanimotoSimilarity(fp, ref_fp) for ref_fp in references)

# If your scored CSV already has a 'Tanimoto' column, you can skip this step
df['Tanimoto'] = df['SMILES'].apply(lambda smi: max_tanimoto(smi, references))
filtered = df[df['Tanimoto'] >= 0.6]

# Drop the 'Input_SMILES' column if it exists
if 'Input_SMILES' in filtered.columns:
    filtered = filtered.drop(columns=['Input_SMILES'])

# ---- Property Filters ----
filtered = filtered[
    (filtered['Molecular weight'] <= 550) &
    (filtered['QED'] >= 0.75) &
    (filtered['SA score'] <= 4) &
    (filtered['SlogP (RDKit)'] >= -1) &
    (filtered['SlogP (RDKit)'] <= 5)
]

print(f"Filtered {len(filtered)} molecules with all property thresholds out of {len(df)} total.")

# Save result
filtered.to_csv('filtered_scoring_results.csv', index=False)


[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerator
[17:05:10] DEPRECATION WARNING: please use MorganGenerat

Filtered 245 molecules with all property thresholds out of 27448 total.


In [4]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from PIL import Image
import numpy as np

# Load molecules
df = pd.read_csv("filtered_scoring_results.csv")
smiles_list = df["SMILES"].tolist()
mols = [Chem.MolFromSmiles(smi) for smi in smiles_list if Chem.MolFromSmiles(smi)]

# Split into chunks of N molecules per page
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

pages = list(chunks(mols, 20))  # 20 mols per page

images = []
for chunk in pages:
    img_array = Draw.MolsToGridImage(
        chunk,
        molsPerRow=4,
        subImgSize=(300, 300),
        useSVG=False,
        returnPNG=False  # Important: this returns a numpy array
    )
    # Convert RDKit image (PIL or array) to actual PIL Image
    pil_img = img_array if isinstance(img_array, Image.Image) else Image.fromarray(np.array(img_array))
    images.append(pil_img.convert("RGB"))

# Save to PDF
if images:
    images[0].save("molecule_grid.pdf", save_all=True, append_images=images[1:])
